In [3]:
# !pip install catboost

In [2]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score #, mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
PATH = 'https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/'

train = pd.read_csv(PATH + 'car_train.csv')
test = pd.read_csv(PATH + 'car_test.csv')
rides = pd.read_csv(PATH + 'rides_info.csv')

rides.sample(1)

,user_id,car_id,ride_id,ride_date,rating,ride_duration,ride_cost,speed_avg,speed_max,stop_times,distance,refueling,user_ride_quality,deviation_normal
433584,w31950390Y,e22337672e,S1I,2020-03-20,4.26,566,9051,46,81.0,2,1401.173106,0,12.11073,-6.698


In [70]:
train['target_class'].unique()

array(['another_bug', 'electro_bug', 'gear_stick', 'engine_fuel',
       'engine_ignition', 'engine_overheat', 'engine_check',
       'wheel_shake', 'break_bug'], dtype=object)

In [47]:
features2drop = ['car_id', 'deviation_normal_count', 'user_ride_quality_median', 'distance_sum', 'year_to_work', 'riders', 'year_to_start', 'car_rating', 'fuel_type'] # то что надо выбросить
targets = ['target_class', 'target_reg']
# cat_features = ['car_type', 'fuel_type', 'model'] # категориальные признаки
cat_features = ['car_type', 'model']

filtered_features = [i for i in train.columns if (i not in targets and i not in features2drop)]
num_features = [i for i in filtered_features if i not in cat_features]

X = train[filtered_features].drop(targets, axis=1, errors='ignore')
y = train['target_class']

In [48]:
f = lambda x: x.nunique()
rides_df_gr = rides.groupby('car_id', as_index=False).agg(mean_rating = ('rating', 'mean'),
                                                   distance_sum = ('distance', 'sum'),
                                                   rating_min = ('rating', 'min'),
                                                   speed_max = ('speed_max', 'max'),
                                                   user_ride_quality_median = ('user_ride_quality', 'median'),
                                                   deviation_normal_count = ('deviation_normal', 'count'),
                                                   user_uniq = ('user_id', f))

def add_featues(df):

    if 'mean_rating' not in df.columns:
        return df.merge(rides_df_gr, on = 'car_id', how = 'left')

    return df


train = add_featues(train)
test = add_featues(test)

filtered_features = [i for i in train.columns if (i not in targets and i not in features2drop)]
num_features = [i for i in filtered_features if i not in cat_features]

X = train[filtered_features].drop(targets, axis=1, errors='ignore')
y = train['target_class']

In [69]:
n_splits = 5
clfs = []
scores = []

# параметры валидации, обучение будет идти на n_splits (фолдах)
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=7575)
for num, (train_index, test_index) in enumerate(kf.split(X, y)):

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    clf = CatBoostClassifier(cat_features=cat_features,
                             random_state=42,
                             iterations=256,
                             learning_rate=0.01,
                             depth=9,
                             l2_leaf_reg=1.2995500754435721e-08,
                             bootstrap_type='Bayesian',
                             random_strength=0.00010495778497690342,
                             bagging_temperature=9.880550666320023,
                             od_type='Iter',
                             od_wait=42)

    clfs.append(clf)  # Сохраняем модель для дальнейшего использования

    clf.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=150, plot=False)

    # инференс модели
    y_pred = clf.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print(f'\n Accuracy: {score}')

    scores.append(score)
    print(f"fold: {num} acc: {score}")

# Считаем среднее и дисперсию по всем фолдам
print("mean accuracy score --", np.mean(scores, dtype="float16"), np.std(scores).round(4))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


0:	learn: 2.1495725	test: 2.1499516	best: 2.1499516 (0)	total: 95.2ms	remaining: 24.3s
150:	learn: 0.6582197	test: 0.7779875	best: 0.7779875 (150)	total: 16.9s	remaining: 11.8s
255:	learn: 0.4644130	test: 0.6372073	best: 0.6372073 (255)	total: 29.2s	remaining: 0us

bestTest = 0.6372073435
bestIteration = 255


 Accuracy: 0.7606837606837606
fold: 0 acc: 0.7606837606837606
0:	learn: 2.1529640	test: 2.1532185	best: 2.1532185 (0)	total: 99.2ms	remaining: 25.3s


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


150:	learn: 0.6787476	test: 0.7516689	best: 0.7516689 (150)	total: 18.1s	remaining: 12.6s
255:	learn: 0.4879735	test: 0.6059515	best: 0.6059515 (255)	total: 30s	remaining: 0us

bestTest = 0.6059515113
bestIteration = 255


 Accuracy: 0.8290598290598291
fold: 1 acc: 0.8290598290598291
0:	learn: 2.1587413	test: 2.1609379	best: 2.1609379 (0)	total: 98.8ms	remaining: 25.2s


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


150:	learn: 0.6582096	test: 0.7948458	best: 0.7948458 (150)	total: 16.5s	remaining: 11.5s
255:	learn: 0.4671343	test: 0.6536451	best: 0.6536451 (255)	total: 28.2s	remaining: 0us

bestTest = 0.6536451054
bestIteration = 255


 Accuracy: 0.7922912205567452
fold: 2 acc: 0.7922912205567452
0:	learn: 2.1495318	test: 2.1502490	best: 2.1502490 (0)	total: 120ms	remaining: 30.6s


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


150:	learn: 0.6672036	test: 0.7698236	best: 0.7698236 (150)	total: 16.9s	remaining: 11.8s
255:	learn: 0.4781878	test: 0.6299286	best: 0.6299286 (255)	total: 28.5s	remaining: 0us

bestTest = 0.6299285653
bestIteration = 255


 Accuracy: 0.7708779443254818
fold: 3 acc: 0.7708779443254818
0:	learn: 2.1683534	test: 2.1719721	best: 2.1719721 (0)	total: 108ms	remaining: 27.6s


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


150:	learn: 0.6695948	test: 0.7804345	best: 0.7804345 (150)	total: 16.8s	remaining: 11.7s
255:	learn: 0.4777732	test: 0.6297351	best: 0.6297351 (255)	total: 28.6s	remaining: 0us

bestTest = 0.6297350542
bestIteration = 255


 Accuracy: 0.7922912205567452
fold: 4 acc: 0.7922912205567452
mean accuracy score -- 0.789 0.0235


In [67]:
print(scores.index(max(scores)))
print(clf.feature_importances_)
print(clf.feature_names_)

for i in range(len(clf.feature_importances_)):
    print(clf.feature_names_[i], '=', clf.feature_importances_[i])

1
[ 0.         11.34922037 31.47604748 13.02922055 35.89507208  8.25043953]
['model', 'car_type', 'mean_rating', 'rating_min', 'speed_max', 'user_uniq']
model = 0.0
car_type = 11.349220366464944
mean_rating = 31.476047477897225
rating_min = 13.02922054770155
speed_max = 35.89507208139442
user_uniq = 8.250439526541832


In [68]:
best_model_id = scores.index(max(scores))
test['target_class'] = clfs[best_model_id].predict(test[filtered_features])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [55]:
test[['car_id', 'target_class']].to_csv('simple_baseline_v10.csv', index=False)